## Tutorial: Cylinder entropy production for different Riemann solvers

We investigate the entropy production on a cylinder surface for different Riemann solvers in a subsonic compressible flow simulation.

In [ ]:
import ngsolve as ngs
from ngsolve.webgui import Draw

# Import necessary modules
from dream.compressible import CompressibleFlowSolver, Initial, FarField, InviscidWall
from dream.mesh import get_cylinder_omesh

ngs.ngsglobals.msg_level = 0
ngs.SetNumThreads(4)

In [ ]:
# Create mesh
mesh = get_cylinder_omesh(1, 8, 32, 16, 1.2)
Draw(mesh)

In [ ]:
# Set configuration
cfg = CompressibleFlowSolver(mesh)
cfg.time = "pseudo_time_stepping"

cfg.fem = "conservative"
cfg.fem.order = 2
cfg.fem.method = "hdg"
cfg.fem.mixed_method = "inactive"
cfg.fem.scheme = "implicit_euler"

cfg.mach_number = 0.3
cfg.equation_of_state = "ideal"
cfg.equation_of_state.heat_capacity_ratio = 1.4
cfg.dynamic_viscosity = "inviscid"
cfg.scaling = "aerodynamic"

cfg.nonlinear_solver = "pardiso"
cfg.nonlinear_solver.method = "newton"
cfg.nonlinear_solver.convergence_criterion = 1e-12
cfg.nonlinear_solver.max_iterations = 300

cfg.optimizations.static_condensation = True

cfg.io.log.to_terminal = True

mesh.Curve(cfg.fem.order)

In [ ]:
# Setup boundary and initial fields
cfg.bcs.clear()
cfg.dcs.clear()

Uinf = cfg.get_farfield_fields((1, 0))

cfg.bcs['cylinder'] = InviscidWall()
cfg.bcs['right|left'] = FarField(Uinf)
cfg.dcs['default'] = Initial(Uinf)

In [ ]:
# Define a decorator for the moving vortex routine
def cylinder_entropy_production_routine(func):

    def wrapper(*args, **kwargs):
        cfg.io.sensor.list.clear()

        cfg.time.timer.interval = (0, 100)
        cfg.time.timer.step = 0.1
        cfg.time.max_time_step = 10
        cfg.time.increment_at = 10
        cfg.time.increment_factor = 10

        # Set logging paths
        cfg.io.path = func.__name__

        func(*args, **kwargs)

        # Initialize and solve the configuration
        cfg.initialize()

        # Plot pressure fluctuations
        Uh = cfg.get_solution_fields()

        s_h = cfg.specific_entropy(Uh)
        s_inf = cfg.specific_entropy(Uinf)
        s_rel = s_h/s_inf - 1

        cfg.io.draw({'s_rel': ngs.IfPos(s_rel, s_rel, -s_rel)}, min=0, max=5e-4)

        with ngs.TaskManager():
            cfg.solve()

    return wrapper

In [ ]:
@cylinder_entropy_production_routine
def lax_friedrich():
    cfg.riemann_solver = "lax_friedrich"
lax_friedrich()

In [ ]:
@cylinder_entropy_production_routine
def hll():
    cfg.riemann_solver = "hll"
hll()


In [ ]:
@cylinder_entropy_production_routine
def hllem():
    cfg.riemann_solver = "hllem"
hllem()

In [ ]:
@cylinder_entropy_production_routine
def roe():
    cfg.riemann_solver = "roe"
roe()

In [ ]:
@cylinder_entropy_production_routine
def upwind():
    cfg.riemann_solver = "upwind"
upwind()